COMP 215 - LAB 3 (NEO)
----------------
#### Name: Watson Li
#### Date: 23 Jan 2023

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [2]:
import datetime, json, requests
from dataclasses import dataclass
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [3]:
today = datetime.date.today()   # get a date object representing today's date
# print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
# print(formatted_date)

### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [4]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')

12 Near Earth Objects found for 2023-01-09


Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [5]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [6]:
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [7]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [8]:
def neomisses(close_approach_data):
    '''parser for close approach data for a single NEO, returns a tuple of the date and miss distance of the closest approach'''
    # object has to be orbiting Earth
    earth_approaches = [item for item in close_approach_data if item['orbiting_body'] == 'Earth']
    # sort the list by miss distance
    earth_approaches.sort(key=lambda x: x['miss_distance']['kilometers'])
    # return the date and miss distance of the closest approach
    return earth_approaches[0]['close_approach_date'], earth_approaches[0]['miss_distance']['kilometers']

# test the function
neomisses(data['close_approach_data'])

('2049-01-09', '10799717.984332507')

## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [9]:
# fetch list of NEOs for the week
today = datetime.date.today()
week = datetime.timedelta(days=7)
weekbefore = today - week
API_KEY = 'DEMO_KEY'  # substitute your API key here
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={weekbefore}&end_date={today}&api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)  # recall json.loads for lab 1
# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects']
def neomisses(close_approach_data):
    '''parser for close approach data for a single NEO, returns a tuple of the date and miss distance of the closest approach'''
    # object has to be orbiting Earth
    earth_approaches = [item for item in close_approach_data if item['orbiting_body'] == 'Earth']
    # sort the list by miss distance
    earth_approaches.sort(key=lambda x: x['miss_distance']['kilometers'])
    # return the date and miss distance of the closest approach
    return earth_approaches[0]['close_approach_date'], earth_approaches[0]['miss_distance']['kilometers']
# make a new list of 2-tuples of dates and miss distances
misslist = [neomisses(dct['close_approach_data']) for item in neos.values() for dct in item]
# make a new list of just the miss distances
missdistances = [float(item[1]) for item in misslist]
# find the index of the minimum miss distance
minindex = missdistances.index(min(missdistances))
# print the date and miss distance of the closest approach
print(f'The closest approach will be on {misslist[minindex][0]} and the miss distance will be {misslist[minindex][1]} kilometers.')

The closest approach will be on 2023-01-20 and the miss distance will be 1993300.692164614 kilometers.


## Lab 3

# Exercise 1: Define an Asteroid class

Using what we learned in the textbook, define a simple Asteroid class with some basic state
variables for a single NEO. Your Asteroid class should define at least 4 "state variables:”
`id, name, estimated_diameter (m), is_potentially_hazardous (bool)`

Provide an `__init__(self, ...)` method to initialize a new Asteroid object with
specific data values and a little code to test your class.

Add a `__str__(self)` method that returns a nicely formatted string representation of the
object, plus a little code to test it.

In [32]:
@dataclass
class Asteroid:
    id: str
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool
    # for exercise 4:
    close_approaches: list

    '''
    def __init__(self, id, name, estimated_diameter, is_potentially_hazardous):
        self.id = id
        self.name = name
        self.estimated_diameter = estimated_diameter
        self.is_potentially_hazardous = is_potentially_hazardous
        self.close_approaches = []

    def __str__(self):
        return f'{self.name} (NEO id: {self.id} - Diameter: {self.estimated_diameter} km - Hazardous: {self.is_potentially_hazardous} - Close Approaches: {self.close_approaches})'
    '''
    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)
        #close_approach_list = CloseApproach.CA_Constructor('close_approach_data', data)
        return cls(data['id'], data['name'], data['estimated_diameter']['kilometers']['estimated_diameter_max'], data['is_potentially_hazardous_asteroid'], [])

# some changes were made here, specified more in comments and code blocks below

@dataclass
class CloseApproach:
    # asteroid: object
    # commented this out as I wasn't sure how have the asteroid object be passed in - as I need 
    # to pass in the CloseApproach object to the Asteroid object again as well, I kept running
    # into a circular import error, so instead I've taken the asteroid parameter out of the 
    # object here. it's still in the version of the CloseApproach class that's in its own code block below, as that
    # was the version i started from. 
    orbiting_body: str
    approach_date: datetime.date
    miss_distance: float # in kilometers
    relative_velocity: float # in kilometers per second

    '''
    had we not been using dataclass
    def __init__(self, orbiting_body, approach_date, miss_distance, relative_velocity):
        # self.asteroid = asteroid
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity

    def __str__(self):
        return f'{self.asteroid} will approach {self.orbiting_body} on {self.approach_date} at a miss distance of {self.miss_distance} kilometers and a relative velocity of {self.relative_velocity} kilometers per second.'
    '''
    @classmethod
    def CA_Constructor(cls, close_approach_data, asteroid):
        neo_id = asteroid.id
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)
        # pprint(data[close_approach_data])
        # wip
        # some of the shit that follows is kinda scuffed but w/e
        CA_list = [cls(i['orbiting_body'], i['close_approach_date'], i['miss_distance']['kilometers'], i['relative_velocity']['kilometers_per_second']) for i in data[close_approach_data]]
        asteroid.close_approaches = CA_list # kind of a hacky way to do this, but it works
        return(asteroid) # this is an entirely different return statement compared to the original, which is in the exercise 3 code block below

# test our code
asteroid = Asteroid.from_NEO('2496816') # literally just yoinked the first asteroid ID that i found when browsing the raw output from the API lol 
asteroid = CloseApproach.CA_Constructor('close_approach_data', asteroid)
pprint(asteroid) #  changed to prettyprint to make it look nicer because my god the output is ugly

Asteroid(id='2496816',
         name='496816 (1989 UP)',
         estimated_diameter=0.459240286,
         is_potentially_hazardous=True,
         close_approaches=[CloseApproach(orbiting_body='Earth',
                                         approach_date='1910-10-01',
                                         miss_distance='63345120.345287499',
                                         relative_velocity='19.7542583014'),
                           CloseApproach(orbiting_body='Earth',
                                         approach_date='1915-12-29',
                                         miss_distance='12065148.878671504',
                                         relative_velocity='8.6494767509'),
                           CloseApproach(orbiting_body='Mars',
                                         approach_date='1928-06-17',
                                         miss_distance='10573160.043623126',
                                         relative_velocity='12.9182561414'),
   

# Exercise 2: Factory method: Asteroid.from_NEO

We want to be able to construct Asteroid objects easily from the NEO API. Define a
`“@classmethod”: from_NEO(cls, neo_id)` that takes the id for a single NEO, fetches the
NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is sometimes called a “Factory” – it constructs an object from raw materials.

In [11]:
# added to code block above

# Exercise 3: Define a CloseApproach class

Each NEO comes with a list of “close_approach_data”, where each record in this list represents
a single “close approach” to another orbiting body.

Develop a class named “CloseApproach” to represent a single close approach record.
State variables are asteroid (Asteroid object), orbiting body (str), approach date
(datetime object!), miss distance (float choose units, document it, and be consistent!),
and relative velocity (ditto).

Define a Factory class method to construct a CloseApproach object from one close
approach data record (a dictionary object). Remember to parse the date/time string into a
datetime object.

In [31]:
# i tweaked this a little and pasted it in the code block above

@dataclass
class CloseApproach:
    asteroid: object
    orbiting_body: str
    approach_date: datetime.date
    miss_distance: float # in kilometers
    relative_velocity: float # in kilometers per second

    '''
    had we not been using dataclass
    def __init__(self, asteroid, orbiting_body, approach_date, miss_distance, relative_velocity):
        self.asteroid = asteroid
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity

    def __str__(self):
        return f'{self.asteroid} will approach {self.orbiting_body} on {self.approach_date} at a miss distance of {self.miss_distance} kilometers and a relative velocity of {self.relative_velocity} kilometers per second.'
    '''
    @classmethod
    def CA_Constructor(cls, close_approach_data, asteroid):
        neo_id = asteroid.id
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)
        # pprint(data[close_approach_data])
        # wip
        # our return will use index 0 of the close_approach_data list, as we're only currently interested in testing
        # our code out with the first close approach
        return(cls(asteroid, data[close_approach_data][0]['orbiting_body'], data[close_approach_data][0]['close_approach_date'], data[close_approach_data][0]['miss_distance']['kilometers'], data[close_approach_data][0]['relative_velocity']['kilometers_per_second']))
        
# test our code
asteroid = Asteroid.from_NEO('2496816')
close_approach_test = CloseApproach.CA_Constructor('close_approach_data', asteroid)
print(close_approach_test)

CloseApproach(asteroid=Asteroid(id='2496816', name='496816 (1989 UP)', estimated_diameter=0.459240286, is_potentially_hazardous=True, close_approaches=[]), orbiting_body='Earth', approach_date='1910-10-01', miss_distance='63345120.345287499', relative_velocity='19.7542583014')


# Exercise 4: Add list of CloseApproaches to Asteroid

Every Asteroid should have a list of “close approaches”. Add a new state variable to your Asteroid
class, initially an empty list. In the factory method, use a list comprehension to build a list of
CloseApproach objects for the Asteroid from the NEO data record.
Extend your test code to demonstrate this new feature.

In [13]:
# code added to code block above